In [1]:
from urllib.request import urlretrieve
from os.path import isfile,isdir
import os
from tqdm import tqdm

In [2]:
vgg_dir = 'F:\Deeplearning\deep-learning\TransferLearningVGG16-FlowerClassification'
# Make sure vgg exists
#if not isdir(vgg_dir):
#    raise Exception("VGG directory doesn't exist!")

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(vgg_dir + "vgg16.npy"):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='VGG16 Parameters') as pbar:
        urlretrieve(
            'https://s3.amazonaws.com/content.udacity-data.com/nd101/vgg16.npy',
            vgg_dir + 'vgg16.npy',
            pbar.hook)
else:
    print("Parameter file already exists!")

Parameter file already exists!


In [3]:
# downloading flower dataset from tensorflow

import tarfile

dataset_folder_path = 'flower_photos'

class DLProgress(tqdm):
    last_block=0
    def hook(self,block_num=1,block_size=1,total_size=None):
        self.total=total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num
if not isfile('flower_photos.tar.gz'):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='Flowers Dataset') as pbar:
        urlretrieve(
            'http://download.tensorflow.org/example_images/flower_photos.tgz',
            'flower_photos.tar.gz',
            pbar.hook)
if not isdir(dataset_folder_path):
    with tarfile.open('flower_photos.tar.gz') as tar:
        tar.extractall()
        tar.close()

Flowers Dataset: 229MB [02:49, 1.35MB/s]                                                                               


In [2]:
import os

import numpy as np
import tensorflow as tf

from tensorflow_vgg import vgg16
from tensorflow_vgg import utils

C:\Users\Lenovo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
data_dir = 'flower_photos/'
contents = os.listdir(data_dir)
classes = [each for each in contents if os.path.isdir(data_dir + each)]

In [ ]:
batch_size = 10
codes_list = []
labels = []
batch = []

codes = None

with tf.Session() as sess:
    vgg=vgg16.Vgg16()
    input_=tf.placeholder(tf.float32,[None,224,224,3])
    with tf.name_scope("content_vgg"):
        vgg.build(input_)
        
    for each in classes:
        print("Starting {} images".format(each))
        class_path = data_dir + each
        files = os.listdir(class_path)
        
        for ii, file in enumerate(files, 1):
            # Add images to the current batch
            # utils.load_image crops the input images for us, from the center
            img = utils.load_image(os.path.join(class_path, file))
            batch.append(img.reshape((1, 224, 224, 3)))
            labels.append(each)
        # Running the batch through the network to get the codes
        if ii % batch_size == 0 or ii == len(files):
            images = np.concatenate(batch)
            print("Len of images {} at batch {}".format(len(images),ii))
            feed_dict = {input_: images}
            codes_batch = sess.run(vgg.relu6, feed_dict=feed_dict)
            
            # Here I'm building an array of the codes
            if codes is None:
                codes = codes_batch
            else:
                codes = np.concatenate((codes, codes_batch))
                
             # Reset to start building the next batch
            batch = []
            print('{} images processed'.format(ii))

F:\Deeplearning\deep-learning\TransferLearningVGG16-FlowerClassification\tensorflow_vgg\vgg16.npy
npy file loaded
build model started
build model finished: 6s
Starting daisy images


C:\Users\Lenovo\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
